### Input Redshift Cluster Endpoint and User
Please input your redshift cluster endpoint and existing user on that cluster.

In [1]:
REDSHIFT_ENDPOINT = 'redshift-cluster.xxxxxxxxxx.us-east-1.redshift.amazonaws.com:5439/dev'
REDSHIFT_USER="awsuser"

### Setup Run SQL function using Redshift Data API to get SQL query output directly into pandas dataframe
In this step, we are creating function run_sql, which we will use to get SQL query output directly into pandas dataframe. We will also use this function to run DDL statements

In [2]:

import boto3
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

session = boto3.session.Session()
region = session.region_name


def run_sql(sql_text):
    client = boto3.client("redshift-data")
    res = client.execute_statement(Database=REDSHIFT_ENDPOINT.split('/')[1], DbUser=REDSHIFT_USER, Sql=sql_text,
                                   ClusterIdentifier=REDSHIFT_ENDPOINT.split('.')[0])
    query_id = res["Id"]
    done = False
    while not done:
        time.sleep(1)
        status_description = client.describe_statement(Id=query_id)
        status = status_description["Status"]
        if status == "FAILED":
            raise Exception('SQL query failed:' + query_id + ": " + status_description["Error"])
        elif status == "FINISHED":
            if status_description['ResultRows']>0:
                results = client.get_statement_result(Id=query_id)
                column_labels = []
                for i in range(len(results["ColumnMetadata"])): column_labels.append(results["ColumnMetadata"][i]['label'])
                records = []
                for record in results.get('Records'):
                    records.append([list(rec.values())[0] for rec in record])
                df = pd.DataFrame(np.array(records), columns=column_labels)
                return df
            else:
                return query_id


## Data Preparation Script
Data preparation script to be run on Redshift

In [32]:
setup_script = """

DROP TABLE IF EXISTS ecommerce_sales CASCADE;
DROP TABLE IF EXISTS ecommerce_sales_data CASCADE;
DROP TABLE IF EXISTS ecommerce_sales_training CASCADE;
DROP TABLE IF EXISTS ecommerce_sales_validation CASCADE;
DROP TABLE IF EXISTS ecommerce_sales_prediction CASCADE;


CREATE TABLE IF NOT EXISTS ecommerce_sales
(
  invoiceno VARCHAR(30)    
  ,stockcode VARCHAR(30)   
  ,description VARCHAR(60)    
  ,quantity DOUBLE PRECISION   
  ,invoicedate VARCHAR(30)    
  ,unitprice    DOUBLE PRECISION
  ,customerid BIGINT    
  ,country VARCHAR(25)    
)
;
 
Copy ecommerce_sales
From 's3://redshift-ml-multiclass/ecommerce_data.txt'
iam_role 'arn:aws:iam::<accountid>:role/RedshiftML' delimiter '\t' IGNOREHEADER 1 region 'us-east-1' maxerror 100;

create table ecommerce_sales_data as (
  select
    t1.stockcode,
    t1.description,
    t1.invoicedate,
    t1.customerid,
    t1.country,
    t1.sales_amt,
    cast(random() * 100 as int) as data_group_id
  from
    (
      select
        stockcode,
        description,
        invoicedate,
        customerid,
        country,
        sum(quantity * unitprice) as sales_amt
      from
        ecommerce_sales
      group by
        1,
        2,
        3,
        4,
        5
    ) t1
);


create table ecommerce_sales_training as (
  select
    a.customerid,
    a.country,
    a.stockcode,
    a.description,
    a.invoicedate,
    a.sales_amt,
    (b.nbr_months_active) as nbr_months_active
  from
    ecommerce_sales_data a
    inner join (
      select
        customerid,
        count(
          distinct(
            DATE_PART(y, cast(invoicedate as date)) || '-' || LPAD(
              DATE_PART(mon, cast(invoicedate as date)),
              2,
              '00'
            )
          )
        ) as nbr_months_active
      from
        ecommerce_sales_data
      group by
        1
    ) b on a.customerid = b.customerid
  where
    a.data_group_id < 80
);

create table ecommerce_sales_validation as (
  select
    a.customerid,
    a.country,
    a.stockcode,
    a.description,
    a.invoicedate,
    a.sales_amt,
    (b.nbr_months_active) as nbr_months_active
  from
    ecommerce_sales_data a
    inner join (
      select
        customerid,
        count(
          distinct(
            DATE_PART(y, cast(invoicedate as date)) || '-' || LPAD(
              DATE_PART(mon, cast(invoicedate as date)),
              2,
              '00'
            )
          )
        ) as nbr_months_active
      from
        ecommerce_sales_data
      group by
        1
    ) b on a.customerid = b.customerid
  where
    a.data_group_id between 80
    and 90
);

create table ecommerce_sales_prediction as (
  select
    customerid,
    country,
    stockcode,
    description,
    invoicedate,
    sales_amt
  from
    ecommerce_sales_data
  where
    data_group_id > 90);  
"""

### Run data preparation script in Redshift

In [33]:
sql_stmt = setup_script.split(";")
for sql_text in sql_stmt[:-1]:
    run_sql(sql_text);

### Run Create Model statement to create a new ML model with Redshift ML

In [13]:
run_sql("""

create model ecommerce_customer_activity
from
  (
select   
  customerid,
  country,
  stockcode,
  description,
  invoicedate,
  sales_amt,
  nbr_months_active  
 from ecommerce_sales_training)
 TARGET nbr_months_active FUNCTION predict_customer_activity
 IAM_ROLE 'arn:aws:iam::<accountid>:role/RedshiftML'
 problem_type MULTICLASS_CLASSIFICATION  
  SETTINGS (
    S3_BUCKET ''redshiftml-<your-account-id>'',
    S3_GARBAGE_COLLECT OFF
  );
""");

### View Model Progress 

In [24]:
df = run_sql("SHOW MODEL ecommerce_customer_activity")
df

,Key,Value
0,Model Name,ecommerce_customer_activity
1,Schema Name,public
2,Owner,demo
3,Creation Time,"Mon, 09.08.2021 14:44:06"
4,Model State,READY
5,Training Job Status,MaxAutoMLJobRuntimeReached
6,validation:accuracy,0.994640
7,Estimated Cost,4.523011
8,,
9,TRAINING DATA:,


## Compare actual vs predicted


In [7]:
df = run_sql("""
select 
 cast(sum(t1.match)as decimal(7,2)) as predicted_matches
,cast(sum(t1.nonmatch) as decimal(7,2)) as predicted_non_matches
,cast(sum(t1.match + t1.nonmatch) as decimal(7,2))  as total_predictions
,predicted_matches / total_predictions as pct_accuracy
from 
(select   
  customerid,
  country,
  stockcode,
  description,
  invoicedate,
  sales_amt,
  nbr_months_active,
  predict_customer_activity(customerid, country, stockcode, description, invoicedate, sales_amt) as predicted_months_active,
  case when nbr_months_active = predicted_months_active then 1
      else 0 end as match,
  case when nbr_months_active <> predicted_months_active then 1
    else 0 end as nonmatch
  from ecommerce_sales_validation
  )t1;
""");
df


,predicted_matches,predicted_non_matches,total_predictions,pct_accuracy
0,35121.00,8373.00,43494.00,0.80749068


## Predict how many customers will qualify for loyalty program


In [8]:
df = run_sql("""
 select 
  customerid,  
  predict_customer_activity(customerid, country, stockcode, description, invoicedate, sales_amt) as predicted_months_active
  from ecommerce_sales_prediction
 where predicted_months_active >=7
 group by 1,2
 limit 10;
""");
df


,customerid,predicted_months_active
0,14031,12
1,14849,11
2,15044,11
3,14156,12
4,18017,10
5,16145,11
6,16670,9
7,14944,8
8,17512,7
9,14051,13
